In [8]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import time
from collections import Counter
import pickle
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib import font_manager, rc  
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)

##### 첫 페이지만 url수집-href

In [9]:
#빈크롬창 열기
driver = webdriver.Chrome(ChromeDriverManager().install())
#연합뉴스: 최신기사
driver.get('https://www.yna.co.kr/news?site=navi_latest_depth01')

url_list=[]

#객체화
soup = bs(driver.page_source, 'html.parser')
url = soup.select('div.list-type038>ul.list>li>div.item-box01>div.news-con>a')
for i in url:
    print('https:' + i['href'])
    url_list.append('https:'+i['href']) 

https://www.yna.co.kr/view/AKR20221002001400091?section=news
https://www.yna.co.kr/view/AKR20221002001500071?section=news
https://www.yna.co.kr/view/AKR20221001053352072?section=news
https://www.yna.co.kr/view/AKR20221001052851099?section=news
https://www.yna.co.kr/view/AKR20221002001300109?section=news
https://www.yna.co.kr/view/AKR20221002001200091?section=news
https://www.yna.co.kr/view/AKR20221002001100071?section=news
https://www.yna.co.kr/view/AKR20221002000551072?section=news
https://www.yna.co.kr/view/AKR20221001053351072?section=news
https://www.yna.co.kr/view/AKR20221001039100004?section=news
https://www.yna.co.kr/view/AKR20221002000700007?section=news
https://www.yna.co.kr/view/AKR20221002000400109?section=news
https://www.yna.co.kr/view/AKR20221002000500072?section=news
https://www.yna.co.kr/view/AKR20221001053700007?section=news
https://www.yna.co.kr/view/AKR20221002000100011?section=news
https://www.yna.co.kr/view/AKR20220930150600053?section=news
https://www.yna.co.kr/vi

##### 두번째 페이지부터 url수집-href

In [10]:
#2페이지부터 URL 수집-href
for i in range(2, 4):
    page =driver.find_element_by_css_selector(f'#container > div > div > div.section01 > section > div.paging.paging-type01 > a:nth-child({i})')
    page.click()
    
    #url_list=[]
    
    #객체화
    soup = bs(driver.page_source, 'html.parser')
    url = soup.select('div.list-type038>ul.list>li>div.item-box01>div.news-con>a')
    
    for i in url:
        print('https:' + i['href'])
        url_list.append('https:'+i['href']) 

https://www.yna.co.kr/view/AKR20221001051900083?section=news
https://www.yna.co.kr/view/AKR20221001042251001?section=news
https://www.yna.co.kr/view/AKR20221001051300061?section=news
https://www.yna.co.kr/view/AKR20221001051600061?section=news
https://www.yna.co.kr/view/AKR20221001051200082?section=news
https://www.yna.co.kr/view/AKR20221001050700530?section=news
https://www.yna.co.kr/view/AKR20221001050200109?section=news
https://www.yna.co.kr/view/AKR20221001040252007?section=news
https://www.yna.co.kr/view/AKR20221001050300007?section=news
https://www.yna.co.kr/view/AKR20220919080561007?section=news
https://www.yna.co.kr/view/AKR20220915170164007?section=news
https://www.yna.co.kr/view/AKR20221001050100002?section=news
https://www.yna.co.kr/view/AKR20221001048500009?section=news
https://www.yna.co.kr/view/AKR20221001050000007?section=news
https://www.yna.co.kr/view/AKR20221001049900007?section=news
https://www.yna.co.kr/view/AKR20221001049700002?section=news
https://www.yna.co.kr/vi

In [11]:
title_list=[]
contents_list=[]
                
for i in url_list:
    driver.get(i)
    time.sleep(1)
    soup = bs(driver.page_source, 'html.parser')
                     
    title=driver.find_elements_by_css_selector('#articleWrap > div.content03 > header > h1')
    for j in title:
        title_list.append(j.text)

    contents = driver.find_elements_by_css_selector('#articleWrap > div.content01.scroll-article-zone01 > div > div > article')
    for j in contents:
        contents_list.append(j.text)
        
    #driver.back()
    time.sleep(1)

In [12]:
len(url_list), len(title_list), len(contents_list)

(75, 75, 75)

In [13]:
import re
result_list =[]
for i in contents_list:
    result = i.replace("\n", "")
    result = result.replace("\'", "")
    result = result.replace("기자 페이지", '')
    
    #정규표현식
    result = re.compile("[a-zA-Z]+[0-9]+@yna+.+[a-zA-Z]+.+[a-zA-Z]+제보는\s카.+").sub("", result)
    result = re.compile("[a-zA-Z]+@yna+.+[a-zA-Z]+.+[a-zA-Z]+제보는\s카.+").sub("", result)
    result = re.compile("[a-zA-Z]+@yna+.+[a-zA-Z]+.+[a-zA-Z]").sub("", result)
    result = re.compile("제보는\s카.+").sub("", result)
    result = re.compile("연합뉴스+,+\s무단.+").sub("", result)
    result = re.compile("연합뉴스가+\s+제공하는+\s+기사.+").sub("", result)
    result = re.compile("[가-힣]+\s+기자").sub("", result)
    result_list.append(result)

In [14]:
type(result_list)

list